# Karar Destek Projesi
## Üniversite Öğrenci İşleri Yönetmeliği Chatbotu

**Öğrenci Ad Soyad:** Yusuf SUKARİ  
**Öğrenci No:** 222803077  
**Ders:** Karar Destek   
**Proje Türü:** RAG ve Fine-Tuning Tabanlı Chatbot

## Video Sunum Linki


https://youtu.be/ddgKg3hobW0


In [ ]:
# NOT: Bu hücre proje için gerekli paketleri kurar.
# NOT: Daha önce ortamda paket kavgası yaşadığım için "pinli" kurulum kullandım.


!python -m pip install -U --no-cache-dir -q \
  "huggingface-hub>=0.34.0,<1.0" \
  "transformers==4.45.2" \
  "tokenizers==0.20.3" \
  "accelerate==0.34.2" \
  "peft==0.13.2" \
  "sentence-transformers==3.0.1" \
  "gradio==4.44.0" \
  sentencepiece

print("✅ Kurulum tamam. Şimdi Kernel/Runtime -> Restart yap.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 152.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 146.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 139.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 199.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 218.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 196.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 require

In [2]:
# NOT: Restart sonrası doğru paket sürümlerini kontrol eder.

import transformers, huggingface_hub, peft, accelerate
import gradio, sentence_transformers

print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("peft:", peft.__version__)
print("accelerate:", accelerate.__version__)
print("sentence-transformers:", sentence_transformers.__version__)
print("gradio:", gradio.__version__)
print("✅ Ortam hazır.")


transformers: 4.45.2
huggingface_hub: 0.36.0
peft: 0.13.2
accelerate: 0.34.2
sentence-transformers: 3.0.1
gradio: 4.44.0
✅ Ortam hazır.


In [ ]:
# NOT: Gemma modeli gated repo olduğu için HuggingFace token gerektirir.

import os
os.environ["HF_TOKEN"] = "Token Anahtari Buraya Girilir"

from huggingface_hub import whoami
print(whoami(token=os.environ["HF_TOKEN"]))


{'type': 'user', 'id': '69487ac54cb4a06b00722136', 'name': 'yassafg4', 'fullname': 'yusuf sukari', 'email': 'yousefsekari96@gmail.com', 'emailVerified': True, 'canPay': False, 'billingMode': 'prepaid', 'periodEnd': 1767225600, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/TdDrvqPPfA3oNHxV0uCoV.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'GA4', 'role': 'write', 'createdAt': '2025-12-28T00:28:07.715Z'}}}


In [ ]:
# NOT: Gemma-2b-it modelini yükler.
# NOT: dtype parametresi yerine torch_dtype kullanıyoruz.

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "google/gemma-2b-it"
tok = os.environ["HF_TOKEN"]

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    token=tok,
    use_fast=False,
    trust_remote_code=False
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=tok,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
base_model.eval()

print("✅ Gemma yüklendi | CUDA:", torch.cuda.is_available())


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

✅ Gemma yüklendi | CUDA: True


In [5]:
# NOT: Verilen prompt ile modelden "sadece cevap" üretir.
# NOT: Model seçimi fonksiyon parametresiyle yapılır (global swap yok, karışıklık olmaz).

def generate_only_answer_with(model_obj, prompt: str, max_new_tokens: int = 80):
    inputs = tokenizer(prompt, return_tensors="pt").to(model_obj.device)
    with torch.no_grad():
        out = model_obj.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
        )
    full = tokenizer.decode(out[0], skip_special_tokens=True)
    if prompt in full:
        full = full.split(prompt, 1)[-1]
    return full.strip()


In [6]:
# NOT: Yönetmelik dokümanı notebook içinde tutulur (dosyadan okuma yok).
# NOT: RAG için metni parçalara (chunk) ayırıyoruz.

import re

university_regulation_text = """
ÜNİVERSİTE ÖĞRENCİ İŞLERİ YÖNETMELİĞİ

1. Devamsızlık
- Öğrenciler, teorik derslerin en az %70’ine devam etmek zorundadır.
- Uygulamalı derslerde devam zorunluluğu %80’dir.
- Devamsızlık sınırını aşan öğrenciler final ve bütünleme sınavına giremez.

2. Sınavlar
- Her ders için bir ara sınav ve bir final sınavı yapılır.
- Final sınavına girebilmek için devamsızlık şartının sağlanması gerekir.
- Final sınavına giremeyen veya başarısız olan öğrenciler bütünleme sınavına girebilir.

3. Bütünleme Sınavı
- Bütünleme sınavı, final sınavının yerine geçen tek bir sınavdır.
- Bütünleme notu, final notu yerine geçer.
- Devamsızlıktan kalan öğrenciler bütünleme sınavına giremez.

4. Mazeret Sınavı
- Sağlık raporu veya resmi belge ile mazeretini belgeleyen öğrencilere mazeret sınavı hakkı tanınır.
- Mazeret sınavı, ara sınavlar için geçerlidir.
- Mazeret sınavına giren öğrenci final sınavına da girer.

5. Not Sistemi
- Başarı notu; ara sınav ve final/bütünleme sınavı notlarının ortalamasıyla hesaplanır.
- Geçme notu CC ve üzeridir.
- FF notu alan öğrenci dersten başarısız sayılır.

6. Ders Tekrarı
- Başarısız olunan dersler, takip eden dönemlerde tekrar alınır.
- Ders tekrarı yapılan derslerde önceki notlar geçersiz sayılır.

7. Disiplin
- Kopya çekmek veya çektirmek disiplin suçudur.
- Disiplin cezası alan öğrenciler hakkında ilgili mevzuat uygulanır.
""".strip()

def chunk_text(text: str, max_chars: int = 700, overlap: int = 120):
    sections = re.split(r"\n(?=\d+\.\s)", text.strip())
    chunks = []
    for sec in sections:
        sec = sec.strip()
        if not sec:
            continue
        if len(sec) <= max_chars:
            chunks.append(sec)
        else:
            start = 0
            while start < len(sec):
                end = min(len(sec), start + max_chars)
                chunks.append(sec[start:end])
                if end == len(sec):
                    break
                start = max(0, end - overlap)
    return chunks

rag_chunks = chunk_text(university_regulation_text)
print("✅ Chunk sayısı:", len(rag_chunks))


✅ Chunk sayısı: 8


In [ ]:
# NOT: RAG için embedding modeli kullanıyoruz.
# NOT: FAISS kullanmıyorum (NumPy 2.x uyumsuzluğu yaşadığım için).
# NOT: Retrieval: cosine similarity = normalize edilmiş vektörlerde dot product.

import numpy as np
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

chunk_embs = emb_model.encode(
    rag_chunks,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

def retrieve_chunks(question: str, top_k: int = 3):
    q_emb = emb_model.encode([question], convert_to_numpy=True, normalize_embeddings=True).astype("float32")[0]
    scores = chunk_embs @ q_emb  # cosine similarity
    top_idx = np.argsort(-scores)[:top_k]
    hits = [{"score": float(scores[i]), "chunk_id": int(i), "chunk": rag_chunks[int(i)]} for i in top_idx]
    return hits

# küçük test
hits = retrieve_chunks("Teorik derslerde devamsızlık sınırı yüzde kaçtır?", top_k=3)
print("✅ Retrieval skorları:", [round(h["score"], 3) for h in hits])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Retrieval skorları: [0.754, 0.716, 0.597]


In [8]:
# NOT: RAG modeli sadece retrieval ile gelen BAĞLAM'ı kullanır.
# NOT: Eğer cevap bağlamda açıkça varsa, BAĞLAM'dan alıp kısa cevap döndürmesi istenir.
# NOT: Bağlamda yoksa sadece "Bu bilgi yönetmelikte yer almamaktadır." döndürür.

SYSTEM_RAG = """
Sen bir üniversite öğrenci işleri asistanısın.

Kurallar:
- Sadece BAĞLAM içindeki bilgiyi kullan.
- Cevap BAĞLAM içinde açıkça geçiyorsa, cevabı BAĞLAM'dan al ve kısa yaz.
- Eğer cevap BAĞLAMDA yoksa SADECE şu cümleyi yaz:
Bu bilgi yönetmelikte yer almamaktadır.
- Cevap dışında hiçbir şey yazma.
""".strip()

def answer_rag(question: str, top_k: int = 4, min_score: float = 0.30):
    hits = retrieve_chunks(question, top_k=top_k)
    if max(h["score"] for h in hits) < min_score:
        return "Bu bilgi yönetmelikte yer almamaktadır."

    context = "\n\n".join([h["chunk"] for h in hits])
    prompt = f"""<system>
{SYSTEM_RAG}
</system>
<user>
BAĞLAM:
{context}

SORU: {question}

CEVAP (yalnızca cevap):
</user>
<assistant>
"""
    return generate_only_answer_with(base_model, prompt, max_new_tokens=70)

print("RAG test:", answer_rag("Teorik derslerde devamsızlık sınırı yüzde kaçtır?"))


/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


RAG test: 10 %
</assistant>


In [9]:
# NOT: Fine-tuning için QA verisi notebook içinde.
# NOT: "Bu bilgi..." örneklerini AZ tutuyoruz; aksi halde model her şeye onu basabiliyor.

train_qa = [
    {"q":"Teorik derslerde devamsızlık sınırı yüzde kaçtır?", "a":"%70"},
    {"q":"Uygulamalı derslerde devam zorunluluğu yüzde kaçtır?", "a":"%80"},
    {"q":"Geçme notu nedir?", "a":"Geçme notu CC ve üzeridir."},
    {"q":"Bütünleme sınavı hangi sınavın yerine geçer?", "a":"Bütünleme sınavı final sınavının yerine geçer."},
    {"q":"Devamsızlık sınırını aşan öğrenciler final sınavına girebilir mi?", "a":"Hayır. Devamsızlık sınırını aşan öğrenciler final sınavına giremez."},
    {"q":"Devamsızlık sınırını aşan öğrenciler bütünleme sınavına girebilir mi?", "a":"Hayır. Devamsızlık sınırını aşan öğrenciler bütünleme sınavına giremez."},
    {"q":"Devamsızlıktan kalan bir öğrenci bütünleme sınavına girebilir mi?", "a":"Hayır. Devamsızlıktan kalan öğrenciler bütünleme sınavına giremez."},
    {"q":"Final sınavına girebilmek için hangi şartın sağlanması gerekir?", "a":"Final sınavına girebilmek için devamsızlık şartının sağlanması gerekir."},
    {"q":"Mazeret sınavına giren bir öğrenci final sınavına da girer mi?", "a":"Evet. Mazeret sınavına giren öğrenci final sınavına da girer."},
    {"q":"Başarısız olunan derslere ne yapılır?", "a":"Başarısız olunan dersler takip eden dönemlerde tekrar alınır."},
    {"q":"Ders tekrarı yapılan derslerde önceki notlar geçerli midir?", "a":"Hayır. Ders tekrarı yapılan derslerde önceki notlar geçersiz sayılır."},
    # Negatif örnek (az)
    {"q":"Yemekhane yemek fiyatları nedir?", "a":"Bu bilgi yönetmelikte yer almamaktadır."},
]
print("✅ Train QA:", len(train_qa))


✅ Train QA: 12


In [10]:
# NOT: datasets/pyarrow kullanmıyoruz.
# NOT: Saf torch Dataset ile fine-tune verisini hazırlıyoruz.

from torch.utils.data import Dataset

SYSTEM_FT = """
Sen bir üniversite öğrenci işleri asistanısın.
Kurallar:
- Yönetmeliğe uygun kısa cevap ver.
- Bilgi yoksa: Bu bilgi yönetmelikte yer almamaktadır.
- Cevap dışında hiçbir şey yazma.
""".strip()

def format_text(q, a):
    return f"""<system>
{SYSTEM_FT}
</system>
<user>
Soru: {q}
</user>
<assistant>
{a}
</assistant>"""

train_texts = [format_text(x["q"], x["a"]) for x in train_qa]

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=512):
        enc = tokenizer(
            texts,
            truncation=True,
            max_length=max_len,
            padding="max_length",
            return_tensors="pt",
        )
        self.input_ids = enc["input_ids"]
        self.attention_mask = enc["attention_mask"]

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        item = {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }
        item["labels"] = item["input_ids"].clone()
        return item

train_ds = TextDataset(train_texts, tokenizer, max_len=512)
print("✅ Train dataset hazır:", len(train_ds))


✅ Train dataset hazır: 12


In [11]:
# NOT: LoRA (PEFT) ile fine-tuning yapıyoruz.
# NOT: Aynı modele ikinci kez LoRA bağlamamak için "ft_model"ı ayrı değişkende tutuyoruz.

import gc
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

ft_model = get_peft_model(base_model, lora_config)
ft_model.print_trainable_parameters()

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

args = TrainingArguments(
    output_dir="gemma_lora_student_affairs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,          # NOT: 1 yerine 3, öğrenme daha iyi olur
    learning_rate=1e-4,          # NOT: biraz daha stabil
    logging_steps=5,
    save_steps=200,
    save_total_limit=1,
    bf16=torch.cuda.is_available(),
    fp16=False,
    report_to=[],
)

trainer = Trainer(
    model=ft_model,
    args=args,
    train_dataset=train_ds,
    data_collator=data_collator,
)

train_out = trainer.train()
train_out


trainable params: 19,611,648 || all params: 2,525,784,064 || trainable%: 0.7765


Step,Training Loss


TrainOutput(global_step=3, training_loss=3.9853006998697915, metrics={'train_runtime': 61.9304, 'train_samples_per_second': 0.581, 'train_steps_per_second': 0.048, 'total_flos': 147566301806592.0, 'train_loss': 3.9853006998697915, 'epoch': 2.0})

In [12]:
# NOT: Fine-tuned model doğrudan (retrieval olmadan) cevap verir.

def answer_finetuned(question: str):
    prompt = f"""<system>
{SYSTEM_FT}
</system>
<user>
Soru: {question}

Cevap (yalnızca cevap):
</user>
<assistant>
"""
    return generate_only_answer_with(ft_model, prompt, max_new_tokens=70)

print("FT test:", answer_finetuned("Geçme notu nedir?"))


/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


FT test: Geçme notu hakkında hiçbir bilgi sunmamışım.
</assistant>


In [13]:
# NOT: Hybrid yaklaşım: RAG ile BAĞLAM getir + Fine-tuned model ile cevap üret.
# NOT: Skor düşükse (alakasızsa) direkt "Bu bilgi..." döndür.

SYSTEM_HYB = """
Sen bir üniversite öğrenci işleri asistanısın.

Kurallar:
- Sadece BAĞLAM içindeki bilgiyi kullan.
- Cevap BAĞLAM içinde açıkça varsa, BAĞLAM'dan al ve kısa yaz.
- BAĞLAMDA yoksa SADECE:
Bu bilgi yönetmelikte yer almamaktadır.
- Cevap dışında hiçbir şey yazma.
""".strip()

def answer_hybrid(question: str, top_k: int = 4, min_score: float = 0.30):
    hits = retrieve_chunks(question, top_k=top_k)
    if max(h["score"] for h in hits) < min_score:
        return "Bu bilgi yönetmelikte yer almamaktadır."

    context = "\n\n".join([h["chunk"] for h in hits])
    prompt = f"""<system>
{SYSTEM_HYB}
</system>
<user>
BAĞLAM:
{context}

SORU: {question}

CEVAP (yalnızca cevap):
</user>
<assistant>
"""
    return generate_only_answer_with(ft_model, prompt, max_new_tokens=70)

print("HYB test:", answer_hybrid("Teorik derslerde devamsızlık sınırı yüzde kaçtır?"))


HYB test: %70
</assistant>


In [14]:
# NOT: Gradio arayüzü:
# - Kullanıcı sadece soru yazar
# - Sistem aynı soruyu 3 modele birden sorar: RAG, FineTuning, Hybrid
# - Model seçimi yok

import gradio as gr

def answer_all_models(question: str):
    question = (question or "").strip()
    if not question:
        return "Lütfen soru giriniz.", "Lütfen soru giriniz.", "Lütfen soru giriniz."

    rag_ans = answer_rag(question, top_k=4, min_score=0.30)
    ft_ans  = answer_finetuned(question)
    hyb_ans = answer_hybrid(question, top_k=4, min_score=0.30)
    return rag_ans, ft_ans, hyb_ans

with gr.Blocks(title="Öğrenci İşleri Chatbot (RAG + FT + Hybrid)") as demo:
    gr.Markdown("# 🎓 Öğrenci İşleri Yönetmeliği Chatbotu")
    gr.Markdown("Tek soruyla **RAG**, **Fine-Tuned**, **Hybrid** cevaplarını aynı anda görürsün.")

    question = gr.Textbox(lines=2, label="Soru", placeholder="Örn: Teorik derslerde devamsızlık sınırı yüzde kaçtır?")
    btn = gr.Button("Cevapla")

    rag_out = gr.Textbox(lines=5, label="RAG Cevabı")
    ft_out  = gr.Textbox(lines=5, label="Fine-Tuned Cevabı")
    hyb_out = gr.Textbox(lines=5, label="Hybrid Cevabı")

    btn.click(answer_all_models, inputs=[question], outputs=[rag_out, ft_out, hyb_out])

    gr.Examples(
        examples=[
            ["Teorik derslerde devamsızlık sınırı yüzde kaçtır?"],
            ["Devamsızlıktan kalan bir öğrenci bütünleme sınavına girebilir mi? Neden?"],
            ["Mazeret sınavına giren bir öğrenci final sınavına da girer mi?"],
            ["Yemekhane yemek fiyatları nedir?"],
        ],
        inputs=question,
    )

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://35cd5a8950855ae275.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
